In [1]:
%load_ext aicrowd.magic

In [24]:
%aicrowd login

02/21/2022 21:19:58 - INFO - root -   Loading config from /home/paulina.knut/.config/aicrowd-cli/config.toml
02/21/2022 21:19:58 - INFO - root -   Config loaded
02/21/2022 21:19:58 - INFO - root -   API Key not provided in parameters, prompting


Please login here: https://api.aicrowd.com/auth/127ukkbHXHDaz2SeHScoiSO_vQEgXJAzdzmnjv_n0ak


02/21/2022 21:20:04 - INFO - root -   Status code 202.
Response: User didn't authenticate yet
02/21/2022 21:20:05 - INFO - root -   Verifying API Key...
02/21/2022 21:20:05 - INFO - root -   API Key verified


API Key valid


02/21/2022 21:20:05 - INFO - root -   Verifying Gitlab access token...
02/21/2022 21:20:06 - INFO - root -   Gitlab access token verified


Gitlab access token valid


02/21/2022 21:20:06 - INFO - root -   Successfully saved config to /home/paulina.knut/.config/aicrowd-cli/config.toml
---
{'aicrowd_api_key': '861ef2bc7c0a0d71f6e2bb50104d2769', 'gitlab': {'oauth_token': 'ca92d04e02fcd8fbccd6b9bac9d73e75544b750733453f07eac9f4244b05348e', 'userid': 22646, 'username': 'paulina_knut1'}}
---


Saved details successfully!


## Imports

In [3]:
import os

import flash
from flash.core.data.io.input import DataKeys
from flash.core.data.utils import download_data
from flash.image import ObjectDetectionData, ObjectDetector

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
# from icevision.core import ClassMap
import numpy as np
import pandas as pd
from pathlib import PosixPath
from PIL import Image
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

In [4]:
# !pip install icevision

In [5]:
# !pip install 'lightning-flash[image]'

## Data

In [20]:
# %aicrowd ds dl -c mask-prediction -o data

train.csv:   0%|          | 0.00/193k [00:00<?, ?B/s]

train.zip:   0%|          | 0.00/318M [00:00<?, ?B/s]

test.zip:   0%|          | 0.00/115M [00:00<?, ?B/s]

sample_submission.csv:   0%|          | 0.00/44.0k [00:00<?, ?B/s]

val.csv:   0%|          | 0.00/82.8k [00:00<?, ?B/s]

val.zip:   0%|          | 0.00/137M [00:00<?, ?B/s]

In [6]:
train_images_root = 'data/train'
train_csv_path = 'data/train.csv'
val_images_root = 'data/val'
val_csv_path = 'data/val.csv'
test_images_root = 'data/test'

In [7]:
class MaskDataset:
    def __init__(self, images_root, csv_file_path) -> None:
        self.images_root = images_root
        self.csv_file_path = csv_file_path
        
        self.image_paths = []
        self.bboxes = []
        self.labels = []
        df = pd.read_csv(self.csv_file_path)
        for i, v in df.iterrows():
            self.image_paths.append(os.path.join(self.images_root, v['ImageID'] + '.jpg'))
            bbox = [int(i) for i in v['bbox'][1:-1].replace(',', '').split(' ')]
            bbox = [{'xmin': bbox[0], 'ymin': bbox[1], 'width': bbox[2] - bbox[0], 'height': bbox[3] - bbox[1]}]
            self.bboxes.append(bbox)
            self.labels.append([v['masktype']])

In [8]:
train_dataset = MaskDataset(train_images_root, train_csv_path)
val_dataset = MaskDataset(val_images_root, val_csv_path)

In [9]:
datamodule = ObjectDetectionData.from_files(
    train_files=train_dataset.image_paths,
    train_targets=train_dataset.labels,
    train_bboxes=train_dataset.bboxes,
    val_files=val_dataset.image_paths,
    val_targets=val_dataset.labels,
    val_bboxes=val_dataset.bboxes,
    transform_kwargs=dict(image_size=(512, 512)),
    batch_size=8,
)

/mnt/ml-team/homes/paulina.knut/aicrowd/venv_pascal02/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:73: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(


## Model

In [13]:
ObjectDetector.available_heads()

['efficientdet', 'faster_rcnn', 'retinanet', 'yolov5']

In [32]:
head = "yolov5"
ObjectDetector.available_backbones()[head]

['extra_large',
 'extra_large_p6',
 'large',
 'large_p6',
 'medium',
 'medium_p6',
 'small',
 'small_p6']

In [40]:
head = "yolov5"  # "yolov5"
backbone = "medium"  # "small"
model = ObjectDetector(head=head, backbone=backbone, num_classes=datamodule.num_classes, image_size=512)

trainer = flash.Trainer(max_epochs=20, gpus=[2])
trainer.finetune(model, datamodule=datamodule, strategy=("freeze_unfreeze", 10))
trainer.save_checkpoint(f"ObjectDetector_{head}_{backbone}.pt")

Using 'yolov5' provided by airctic/IceVision (https://github.com/airctic/icevision) and Ultralytics/YOLOV5 (https://github.com/ultralytics/yolov5).


  0%|          | 0.00/40.7M [00:00<?, ?B/s]

02/22/2022 07:52:26 - INFO - yolov5.models.yolo -   Overriding model.yaml nc=80 with nc=4
02/22/2022 07:52:26 - INFO - yolov5.models.yolo -   
                 from  n    params  module                                  arguments                     
02/22/2022 07:52:26 - INFO - yolov5.models.yolo -     0                -1  1      5280  yolov5.models.common.Conv               [3, 48, 6, 2, 2]              
02/22/2022 07:52:26 - INFO - yolov5.models.yolo -     1                -1  1     41664  yolov5.models.common.Conv               [48, 96, 3, 2]                
02/22/2022 07:52:26 - INFO - yolov5.models.yolo -     2                -1  2     65280  yolov5.models.common.C3                 [96, 96, 2]                   
02/22/2022 07:52:26 - INFO - yolov5.models.yolo -     3                -1  1    166272  yolov5.models.common.Conv               [96, 192, 3, 2]               
02/22/2022 07:52:26 - INFO - yolov5.models.yolo -     4                -1  4    444672  yolov5.models.common.C3   

02/22/2022 07:52:26 - INFO - yolov5.models.yolo -     6                -1  6   2512896  yolov5.models.common.C3                 [384, 384, 6]                 
02/22/2022 07:52:26 - INFO - yolov5.models.yolo -     7                -1  1   2655744  yolov5.models.common.Conv               [384, 768, 3, 2]              
02/22/2022 07:52:26 - INFO - yolov5.models.yolo -     8                -1  2   4134912  yolov5.models.common.C3                 [768, 768, 2]                 
02/22/2022 07:52:26 - INFO - yolov5.models.yolo -     9                -1  1   1476864  yolov5.models.common.SPPF               [768, 768, 5]                 
02/22/2022 07:52:26 - INFO - yolov5.models.yolo -    10                -1  1    295680  yolov5.models.common.Conv               [768, 384, 1, 1]              
02/22/2022 07:52:26 - INFO - yolov5.models.yolo -    11                -1  1         0  torch.nn.modules.upsampling.Upsample    [None, 2, 'nearest']          
02/22/2022 07:52:26 - INFO - yolov5.models.yol

Validation sanity check: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/mnt/ml-team/homes/paulina.knut/aicrowd/venv_pascal02/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(


In [41]:
test_images_paths = [os.path.join(test_images_root, p) for p in os.listdir(test_images_root)]
test_datamodule = ObjectDetectionData.from_files(
    predict_files=test_images_paths,
    transform_kwargs={"image_size": 512},
    batch_size=8,
)
predictions = trainer.predict(model, datamodule=test_datamodule)

/mnt/ml-team/homes/paulina.knut/aicrowd/venv_pascal02/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:73: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/mnt/ml-team/homes/paulina.knut/aicrowd/venv_pascal02/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 700it [00:00, ?it/s]

In [42]:
label_to_name = {i: n for i, n in enumerate(datamodule.labels)}

In [43]:
def generate_submission(preds):
    frame = pd.DataFrame(columns=["ImageID", "bbox", "masktype"])
    for batch in preds:
        for pred in batch:
            img_path = pred[DataKeys.INPUT][DataKeys.INPUT]
            if len(pred[DataKeys.PREDS]['bboxes']) == 0:
                bbox = {'xmin':0, 'ymin':0, 'width':1, 'height':1}
                label = 0
            else:
                bbox = pred[DataKeys.PREDS]['bboxes'][0]
                label = pred[DataKeys.PREDS]['labels'][0]
            bbox = f"[{int(bbox['xmin'])}, {int(bbox['ymin'])}, {int(bbox['xmin'] + bbox['width'])}, {int(bbox['ymin'] + bbox['height'])}]"
            if label == 0:
                mask = 'surgical'
            else:
                mask = label_to_name[label]
            frame.loc[len(frame), :] = {"ImageID": img_path.split('/')[-1].split('.')[0], "bbox": bbox, "masktype": mask}
    return frame

In [44]:
test_df = generate_submission(predictions)

## Submission

In [45]:
!rm -rf assets
!mkdir assets
test_df.to_csv(os.path.join("assets", "submission.csv"), index=False)

In [46]:
%aicrowd notebook submit -c mask-prediction -a assets --no-verify

02/22/2022 09:15:53 - INFO - root -   Loading config from /home/paulina.knut/.config/aicrowd-cli/config.toml
02/22/2022 09:15:53 - INFO - root -   Config loaded


<IPython.core.display.Javascript object>

Using notebook: flash.ipynb for submission...
Removing existing files from submission directory...
Scrubbing API keys from the notebook...


02/22/2022 09:15:57 - INFO - root -   [metachallenge?] Got redirected to /challenges/ai-blitz-xiii/problems/mask-prediction/submissions


Output()

╭─────────────────────────╮                                                   
                                                  │ Successfully submitted! │                                                   
                                                  ╰─────────────────────────╯

                                                        Important links                                                         
┌──────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  This submission │ https://www.aicrowd.com/challenges/ai-blitz-xiii/problems/mask-prediction/submissions/175047              │
│                  │                                                                                                           │
│  All submissions │ https://www.aicrowd.com/challenges/ai-blitz-xiii/problems/mask-prediction/submissions?my_submissions=true │
│                  │                                                                                                           │
│      Leaderboard │ https://www.aicrowd.com/challenges/ai-blitz-xiii/problems/mask-prediction/leaderboards                    │
│                  │                                                                                                           │
│ Discussion forum │ https://discourse.aicrowd.com/c/ai-blitz-xiii                                                             │
│                  │                                                                                                           │
│   Challenge page │ https://www.aicrowd.com/challenges/ai-blitz-xiii/problems/mask-prediction                                 │
└──────────────────┴───────────────────────────────────────────────────────────────────────────────────────────────────────────┘